In [2]:
import requests
import datetime
import matplotlib.pyplot as plt
# import plotly.graph_objects as go
import plotly.offline as py
import plotly
from plotly.offline import init_notebook_mode
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

py.init_notebook_mode(connected=True)
import numpy as np

#serie de mares de ubatuba em 1981
link = "https://www.ime.usp.br/~pam/h-ubatuba81"
f = requests.get(link)

Fs = 150.0;  # sampling rate
Ts = 1.0/Fs; # sampling interval

a = f.text.split("\n")
a = a[:-1]
a = list(map(int, a))

n = len(a) # length of the signal

x = np.arange(n)
y = np.asarray(a)

T = n/Fs

frq = x/T

data=[go.Scatter(x=x, y=y, mode='lines')]
py.iplot(data)

Y = np.fft.fft(y)/n

data=[go.Scatter(x=frq, y=abs(Y), mode='lines')]
py.iplot(data)

In [45]:
from PyEMD import EMD, CEEMDAN
import numpy as np
import pandas as pd

df = pd.read_csv('./data/PETR4.SA.csv')

np_data = np.asarray(df['Adj Close'])
np_data_x = np.asarray(df['Volume'])

# s = np.append (np_data, [26.60, 26.74, 26.90, 27.12, 27.53])
# s = np.append (np_data, [28.60, 25.4, 29.5, 26.04, 30.6, 24.4])
# s = np.append (np_data, [32.04, 33.6, 35.4, 36.07, 38.9, 40])
# s = np.append (np_data, [32.04, 33.6, 35.4, 36.07, 38.9])
s = np.append (np_data, [26.60, 26.74, 26.90])
# s = np_data

data=[go.Scatter(x=np.arange(len(s)), y=s, mode='lines')]

emd_config = {"std_thr": 0.001, "range_thr": 0.005}
emd = EMD(spline_kind='cubic', nbsym=10, **emd_config)

ceemdan_config = {"range_thr": 0.0001, "total_power_thr": 0.001}
ceemdan = CEEMDAN(trials=500, epsilon=2, ext_EMD=emd, parallel=True, **ceemdan_config)

# ceemdan = CEEMDAN()
IMFs = ceemdan(s)
# IMFs = emd(s)
qtd_comp = len(IMFs)
print(f"Quantidade de componentes : {qtd_comp}")

for component in IMFs:
    component_length = len(component)
    data +=[go.Scatter(x=np.arange(component_length), y=component, mode='lines')]

py.iplot(data)

Quantidade de componentes : 6


In [8]:
data=[go.Scatter(x=np.arange(len(s)), y=s, mode='lines')]

emd_config = {"std_thr": 0.001, "range_thr": 0.005}
emd = EMD(spline_kind='cubic', nbsym=10, **emd_config)

ceemdan_config = {"range_thr": 0.0001, "total_power_thr": 0.001}
ceemdan = CEEMDAN(trials=500, epsilon=0.005, ext_EMD=emd, parallel=True, **ceemdan_config)

# ceemdan = CEEMDAN()
IMFs = ceemdan(s)
# IMFs = emd(s)
qtd_comp = len(IMFs)
print(f"Quantidade de componentes : {qtd_comp}")

for component in IMFs:
    component_length = len(component)
    data +=[go.Scatter(x=np.arange(component_length), y=component, mode='lines')]

py.iplot(data)

Quantidade de componentes : 5


In [94]:
length_test = 30
list_length_adjuster = -length_test
# ARIMA
from statsmodels.tsa.arima_model import ARIMA
from random import random
# contrived dataset
# np_data = [x + random() for x in range(1, 100)]
# fit model

data_arima = IMFs[1]
data_arima = np_data
print(data_arima)
model = ARIMA(data_arima[:list_length_adjuster], order=(2, 1, 2))
model_fit = model.fit(disp=False)
print(f"\n----------\n")
print(data_arima[list_length_adjuster:])
# make prediction
yhat = model_fit.predict(len(data_arima), len(data_arima), typ='levels')
print(yhat)

data_arima_plot = []
data_arima_plot +=[go.Scatter(x=np.arange(len(data_arima[:list_length_adjuster])), y=data_arima[:list_length_adjuster], mode='lines')]
data_arima_plot +=[go.Scatter(x=np.arange(len(data_arima[list_length_adjuster:])), y=data_arima[list_length_adjuster:], mode='lines')]
data_arima_plot +=[go.Scatter(x=np.arange(len(yhat)), y=yhat, mode='lines')]

py.iplot(data_arima_plot)

#calculating error mse
mse = np.mean((data_arima[list_length_adjuster:] - yhat[:-1])**2)

print(f"ARIMA mse = {mse}")

[23.765604 24.348383 24.417526 24.802757 24.654589 25.168226 24.95092
 24.684223 24.545937 24.526182 24.516304 24.852144 25.089207 25.217615
 24.82251  25.118839 25.227497 24.466917 25.059574 25.306515 25.267004
 25.48431  25.711498 25.760887 25.178106 24.782999 24.82251  24.536058
 25.405291 25.731253 26.620241 26.511587 26.432568 27.044979 26.709141
 27.064734 26.798038 26.373302 26.254768 26.74865  26.728895 26.373302
 26.432568 26.442444 26.343668 27.410452 27.163511 27.75617  27.84507
 27.904335 28.388336 28.84271  28.615522 28.210539 26.66963  27.005468
 28.279684 27.005468 27.71666  27.71666  27.657393 27.943846 27.203022
 28.121641 28.427851 28.892099 28.8032   28.427851 27.657393 25.513945
 25.612722 26.393053 26.422688 27.262287 27.104246 27.341309 27.262287
 27.459841 26.935459 27.054075 26.797075 26.421459 26.540077 26.460999
 26.045847 27.054075 26.520306 26.372038 25.60104  25.699886 25.581272
 24.978313 24.395124 25.225428 26.184231 26.085011 25.638519 25.88657
 26.0354 

ARIMA mse = 3.3146232652319356


In [91]:
length_test = 30
list_length_adjuster = -length_test

# SARIMAX example
from statsmodels.tsa.statespace.sarimax import SARIMAX
from random import random
# contrived dataset
data_sarimax = np_data
print(data_arima)
data1 = data_sarimax[:list_length_adjuster]
data2 = np_data_x[:list_length_adjuster]
# fit model
model = SARIMAX(data1, exog=data2, order=(6, 1, 2), seasonal_order=(1, 0, 1, 1))
model_fit = model.fit(disp=False)
# make prediction
exog2 = np_data_x[list_length_adjuster:]
yhat = model_fit.predict(len(data1)-1, len(data_sarimax)-1, exog=list(map(list, zip(*[exog2]))))
print(f"yhat = {yhat}")

data_arima_plot = []
data_arima_plot +=[go.Scatter(x=np.arange(len(data_sarimax[:list_length_adjuster])), y=data_sarimax[:list_length_adjuster], mode='lines')]
data_arima_plot +=[go.Scatter(x=np.arange(len(data_sarimax[list_length_adjuster:])), y=data_sarimax[list_length_adjuster:], mode='lines')]
data_arima_plot +=[go.Scatter(x=np.arange(len(yhat)), y=yhat, mode='lines')]

py.iplot(data_arima_plot)

#calculating error mse
mse = np.mean((data_sarimax[list_length_adjuster:] - yhat[:-1])**2)

print(f"SARIMAX mse = {mse}")

[23.765604 24.348383 24.417526 24.802757 24.654589 25.168226 24.95092
 24.684223 24.545937 24.526182 24.516304 24.852144 25.089207 25.217615
 24.82251  25.118839 25.227497 24.466917 25.059574 25.306515 25.267004
 25.48431  25.711498 25.760887 25.178106 24.782999 24.82251  24.536058
 25.405291 25.731253 26.620241 26.511587 26.432568 27.044979 26.709141
 27.064734 26.798038 26.373302 26.254768 26.74865  26.728895 26.373302
 26.432568 26.442444 26.343668 27.410452 27.163511 27.75617  27.84507
 27.904335 28.388336 28.84271  28.615522 28.210539 26.66963  27.005468
 28.279684 27.005468 27.71666  27.71666  27.657393 27.943846 27.203022
 28.121641 28.427851 28.892099 28.8032   28.427851 27.657393 25.513945
 25.612722 26.393053 26.422688 27.262287 27.104246 27.341309 27.262287
 27.459841 26.935459 27.054075 26.797075 26.421459 26.540077 26.460999
 26.045847 27.054075 26.520306 26.372038 25.60104  25.699886 25.581272
 24.978313 24.395124 25.225428 26.184231 26.085011 25.638519 25.88657
 26.0354 

/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



yhat = [26.52970254 26.30771992 26.31782635 26.43586984 26.25668703 26.40318096
 26.52463414 26.4988391  26.35996752 26.43068311 26.4866207  26.50088893
 26.45251675 26.41635305 26.42196925 26.40688073 26.439219   26.50424696
 26.24712287 26.48456876 26.39687811 26.49078616 26.33433054 26.47964082
 26.38300596 26.41370555 26.59594331 26.44967179 26.40739264 26.44581041
 26.44626835]


SARIMAX mse = 1.830734023592658
